In [ ]:
%load_ext autoreload
%autoreload 2

## Exploring the wiki dump

In [137]:
# from wiki.dump import WikiDump
import wiki

path_base = '/Users/harangju/Developer/data/wiki/'
name_xml = 'enwiki-20190801-pages-articles-multistream.xml.bz2'
name_index = 'enwiki-20190801-pages-articles-multistream-index.txt.bz2'
path_xml = path_base + name_xml
path_index = path_base + name_index
dump = wiki.Dump(path_xml, path_index)

In [138]:
%time dump.load_page('Portal:Physics/Topics')
dump.links[:5]

Dump: Loading index...
Dump: Loaded.
CPU times: user 1min 17s, sys: 6.18 s, total: 1min 23s
Wall time: 1min 25s


['Classical physics', 'Mechanics', 'Optics', 'Electricity', 'Magnetism']

In [166]:
dump.load_page('Danielle Bassett')
dump.links[:5]

['University of pennsylvania',
 'Pennsylvania state university',
 'University of cambridge',
 'Sloan research fellowship',
 'Macarthur fellowship']

In [ ]:
dump.load_page('Matter', filter_top=True).strip_code()[:200]

## Get index of physics articles

* [all indices on Wikipedia](https://en.wikipedia.org/wiki/Portal:Contents/Indices)
* topics not searched
* international trade ("topics"), theory of constraints (small)
* too big: mathematics, neuroscience

In [160]:
# natural & physical sciences
topics = ['anatomy', 'biochemistry', 'cognitive science', 'evolutionary biology',
          'genetics', 'immunology', 'molecular biology']
topics += ['chemistry', 'biophysics', 'energy', 'optics', 
           'earth science', 'geology', 'meteorology']
# philosophy
# topics += []
topics += ['philosophy of language', 'philosophy of law', 
           'philosophy of mind', 'philosophy of science']
# social sciences
topics += ['economics', 'accounting', 'education', 'linguistics', 'law', 'psychology', 'sociology']
# technology & applied sciences
topics += ['electronics', 'software engineering', 'robotics']

In [161]:
links = {}
for topic in topics:
    dump.load_page('Index of %s articles' % topic)
    links[topic] = [str(l) for l in dump.article_links]
    print('Topic "' + topic + '" has ' + str(len(links[topic])) + ' articles.')

Topic "anatomy" has 2331 articles.
Topic "biochemistry" has 1216 articles.
Topic "cognitive science" has 127 articles.
Topic "evolutionary biology" has 287 articles.
Topic "genetics" has 1441 articles.
Topic "immunology" has 572 articles.
Topic "molecular biology" has 507 articles.
Topic "chemistry" has 1088 articles.
Topic "biophysics" has 773 articles.
Topic "energy" has 158 articles.
Topic "optics" has 386 articles.
Topic "earth science" has 135 articles.
Topic "geology" has 116 articles.
Topic "meteorology" has 761 articles.
Topic "philosophy of language" has 275 articles.
Topic "philosophy of law" has 208 articles.
Topic "philosophy of mind" has 109 articles.
Topic "philosophy of science" has 448 articles.
Topic "economics" has 562 articles.
Topic "accounting" has 154 articles.
Topic "education" has 872 articles.
Topic "linguistics" has 420 articles.
Topic "law" has 3657 articles.
Topic "psychology" has 1801 articles.
Topic "sociology" has 772 articles.
Topic "electronics" has 127

In [ ]:
links['physics'] = []
for letter in ['!$@', '0–9'] + list(string.ascii_uppercase):
    dump.load_page('Index of physics articles (%s)' % letter)
    links['physics'].extend([str(l) for l in dump.article_links])
print('Topic "' + 'physics' + '" has ' + str(len(links['physics'])) + ' articles.')

## Build graphs of topics

### 1 network per topic

In [ ]:
import networkx as nx

graphs = {}
page_noload = {}
depth = 2
for topic in links.keys():
    print('Graph topic: ' + topic)
    graphs[topic] = nx.DiGraph()
    page_noload[topic] = wiki.Crawler.bfs(graphs[topic], dump, links[topic],
                                          depth_goal = depth, nodes = links[topic])
    path_save = path_base + 'graphs/' + topic + '_d' + str(depth) + '.gexf'
    nx.write_gexf(graphs[topic], path_save)

In [ ]:
for topic in graphs.keys():
    graphs[topic].remove_nodes_from(nx.isolates(graphs[topic]))
    path_save = path_base + 'graphs/' + topic + '_d' + str(depth) + '.gexf'
    nx.write_gexf(graphs[topic], path_save)

Gephi notes
* node size, fruchterman reingold = [10, 40], force atlas 2 = [4 16]
* text size = [1 1.4]
* preview font size = 5

### 1 network for all topics
i.e., connect all networks into 1 big network

In [ ]:
big_list = [item for sublist in list(links.values()) for item in sublist]
big_graph = nx.DiGraph()
wiki.Crawler.bfs(big_graph, dump, big_list, depth_goal = 2, nodes = big_list)
nx.write_gexf(big_graph, path_base + 'graphs/big_graph.gexf')

## Network measures
* degree distribution - homogenous=more cross link?
* modularity - of topics within a subject
* clustering
* core-periphery
* characteristic path length

In [188]:
import networkx as nx

path_saved = '/Users/harangju/Box Sync/Research/my papers/wikipedia paper/data/prelim data/'
graphs = {}
for topic in topics:
    graphs[topic] = nx.read_gexf(path_saved + topic + '_d2.gexf')
graphs

{'anatomy': <networkx.classes.digraph.DiGraph at 0x7fb298315400>,
 'biochemistry': <networkx.classes.digraph.DiGraph at 0x7fb260f20ba8>,
 'cognitive science': <networkx.classes.digraph.DiGraph at 0x7fb30b92f4e0>,
 'evolutionary biology': <networkx.classes.digraph.DiGraph at 0x7fb2b8432080>,
 'genetics': <networkx.classes.digraph.DiGraph at 0x7fb29831c470>,
 'immunology': <networkx.classes.digraph.DiGraph at 0x7fb2fee1d0b8>,
 'molecular biology': <networkx.classes.digraph.DiGraph at 0x7fb2fd0ddf28>,
 'chemistry': <networkx.classes.digraph.DiGraph at 0x7fb276218e10>,
 'biophysics': <networkx.classes.digraph.DiGraph at 0x7fb30b8ff080>,
 'energy': <networkx.classes.digraph.DiGraph at 0x7fb30d0339b0>,
 'optics': <networkx.classes.digraph.DiGraph at 0x7fb30d035e48>,
 'earth science': <networkx.classes.digraph.DiGraph at 0x7fb2fc8c7240>,
 'geology': <networkx.classes.digraph.DiGraph at 0x7fb287a39358>,
 'meteorology': <networkx.classes.digraph.DiGraph at 0x7fb2fd0c4550>,
 'philosophy of langu

In [165]:
import numpy as np
import networkx.algorithms as nxalg
import bct

stats = {}
for topic, graph in graphs.items():
    print('Topic: ' + topic)
    am = nx.convert_matrix.to_numpy_array(graph)
    stats[topic] = {'indegree': {'values': [d[1] for d in graph.in_degree]},
                    'outdegree': {'values': [d[1] for d in graph.out_degree]},
                    'clustering': {'values': list(nxalg.cluster.clustering(graph).values())},
                    'centrality': {'values': list(nxalg.betweenness_centrality(graph).values())},
                    'coreness': {'values': bct.core_periphery_dir(am)[1]},
                    'path length': {'values': [y for x in list(nx.shortest_path_length(graph))
                                               for y in list(x[1].values())]},
                    'char path length': {'values': bct.charpath(am)[0]}}

Topic: anatomy


/anaconda3/envs/wikinet/lib/python3.7/site-packages/bct/algorithms/distance.py:155: RuntimeWarning: divide by zero encountered in true_divide
  efficiency = np.mean(1 / Dv)


Topic: biochemistry
Topic: cognitive science
Topic: evolutionary biology
Topic: genetics
Topic: immunology
Topic: molecular biology
Topic: chemistry
Topic: biophysics
Topic: energy
Topic: optics
Topic: earth science
Topic: geology
Topic: meteorology
Topic: philosophy of language
Topic: philosophy of law
Topic: philosophy of mind
Topic: philosophy of science
Topic: economics
Topic: accounting
Topic: education
Topic: linguistics
Topic: law
Topic: psychology
Topic: sociology
Topic: electronics
Topic: software engineering
Topic: robotics


In [212]:
import scipy.io as sio

path = '/Users/harangju/Developer/data/wiki/graphs/'
for topic, graph in graphs.items():
    x = nx.convert_matrix.to_numpy_array(graph)
    sio.savemat(path + topic + '.mat', {'X': x})

In [220]:
ex_graph = nx.DiGraph()
ex_graph.add_edge(0,1)
ex_graph.add_edge(2,2)
ex_graph.add_edge(3,4)
nx.convert_matrix.to_numpy_array(ex_graph)

array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0.]])

converting to numpy is row-to-column

In [274]:
metrics = sio.loadmat(path + 'metrics.mat')
for i in range(len(metrics['topics'][0])):
    topic = metrics['topics'][0][i][0]
    stats[topic]['rich club'] = {'values': [x for x in metrics['rich'][0][i][0] 
                                            if not np.isnan(x)]}
    stats[topic]['modularity'] = {'values': [x[0] for x in metrics['mod'][0][i]]}
    stats[topic]['controllability'] = {'values': [x[0] for x in metrics['cont'][0][i]]}
    stats[topic]['observability'] = {'values': [x[0] for x in metrics['obs'][0][i]]}

In [275]:
for topic in stats.keys():
    for stat in stats[topic].keys():
        if 'values' in stats[topic][stat].keys():
            stats[topic][stat]['mean'] = np.mean(stats[topic][stat]['values'])
            stats[topic][stat]['std'] = np.std(stats[topic][stat]['values'])

/anaconda3/envs/wikinet/lib/python3.7/site-packages/numpy/core/_methods.py:117: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


In [276]:
import pandas

mean = {}
for k, v in stats.items():
    mean[k] = {}
    for kk, vv in v.items():
        mean[k][kk] = vv['mean']
pandas.DataFrame(mean).transpose()

,indegree,outdegree,clustering,centrality,coreness,path length,char path length,rich club,modularity,controllability,observability
anatomy,3.619661,3.619661,0.097954,0.001512,0.737638,6.333171,0.002113,0.115980,0.604342,1.001692,1.001692
biochemistry,4.508869,4.508869,0.096448,0.001708,0.796979,4.279866,0.004946,0.211186,0.478762,1.001843,1.001843
cognitive science,2.750000,2.750000,0.165694,0.008716,0.753184,3.090568,0.026970,0.339975,0.428808,1.005851,1.005851
evolutionary biology,3.418803,3.418803,0.127987,0.004773,0.771078,3.879429,0.014343,0.365470,0.407169,1.002203,1.002203
genetics,3.811458,3.811458,0.092073,0.001483,0.875016,4.987032,0.003926,inf,0.498293,1.000889,1.000889
immunology,2.238095,2.238095,0.117715,0.001913,0.828653,4.329856,0.005747,0.279958,0.530708,1.001612,1.001612
molecular biology,3.191617,3.191617,0.103677,0.002207,0.744927,4.401425,0.009396,inf,0.477176,1.000954,1.000953
chemistry,5.779268,5.779268,0.111526,0.002100,0.729179,4.312141,0.007022,0.242642,0.462795,1.001251,1.001251
biophysics,1.519231,1.519231,0.066331,0.000897,0.753527,4.638013,0.004720,0.266737,0.695900,1.001853,1.001853
energy,1.651786,1.651786,0.112055,0.000664,0.745879,1.880531,0.014640,0.371056,0.519967,1.000400,1.000400


### Local structures

**Indegree**
This is mostly a function of how Wikipedians revised the document and should largely be uniform across pages. The large values are likely pages with 'lists' of links.

**Outdegree**
This is 1st-order measure of an idea's influence.

### Mesoscale structures

**Clustering**
These look equally clustered among the topics.

**Centrality**
This reveals the distribution of sources of ideas within a field.

**Path lengths**

**Rich-club coefficient**

**Modularity**

**Controllability**
This is an nth-order measure of influence.

**Observability**
This is an nth-order measure of the inverse of influence.

**Coreness**
It seems that the more focused a topic is on a subtopic, the stronger the coreness. For example, genetics is heavily focused on DNA, and so it has high coreness. At the same time, in the field of economics, the concept of "economics" has high degree. Yet, it has low coreness because the field itself is heterogeneous, with major subfields such as "macroeconomics" and "microeconomics".

**Characteristic path length**
I'm not sure what path length reveals. Perhaps it is a measure of the heterogeneity in research? It describes how far one idea is to another, topologically. Cognitive science and earth science have ideas that are far away.

### Plot

* nice plots [seaborn](https://seaborn.pydata.org/examples/index.html)
* interactive [Bokeh](https://bokeh.pydata.org/en/latest/docs/gallery.html#gallery)

In [206]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='whitegrid')

def plot_violin(data, xlabels, title, save_dir=None):
    f, ax = plt.subplots(figsize=(30, 6))
    data = [x[stat]['values'] for x in stats.values()]
    sns.violinplot(data=data)
    plt.xticks(np.arange(len(stats)), xlabels, rotation='vertical', fontsize=24)
    plt.yticks(fontsize=24)
    plt.title(title, fontsize=36)
    plt.subplots_adjust(bottom=0.2)
    sns.despine(left=True, bottom=True)
    if save_dir:
        plt.savefig(save_dir, dpi=300)
    return f, ax

def scatter(data, xlabels, title, save_dir=None):
    f, ax = plt.subplots(figsize=(30, 6))
    sns.scatterplot(x=np.arange(len(stats)), y=data, marker='x', s=100)
    plt.xticks(np.arange(len(stats)), xlabels, rotation='vertical', fontsize=24)
    plt.yticks(fontsize=24)
    plt.title(title, fontsize=24)
    sns.despine(left=True, bottom=True)
    if save_dir:
        plt.savefig(path_saved + stat, dpi=300)

In [ ]:
for stat in ['indegree', 'outdegree', 'clustering', 'centrality', 'path length', #'rich club', 
             'modularity', 'controllability', 'observability']:
    data = [x[stat]['values'] for x in stats.values()]
    plot_violin(data, stats.keys(), stat)

In [209]:
for stat in ['coreness', 'char path length']:
    data = [x[stat]['values'] for x in stats.values()]
    scatter(data, stats.keys(), stat)